# Задание
Менеджеры часто не имеют времени глубоко вникать во все цифры отчета. Им нужна краткая текстовая сводка с основными выводами: что выросло, что упало, и насколько это значимо. Поэтому нужно автоматизировать создание таких сводок с помощью LLM

Предоставляются ключевые показатели за период в виде словаря. Данные включают значения метрик и их изменение по сравнению с предыдущим периодом.

1) Написать функцию на Python, которая принимает на вход структуру данных с KPI.
2) Используя любую доступную вам LLM (OpenAI API, Hugging Face, Ollama и т.д.), составить промпт (инструкцию) для модели.
3) Промпт должен инструктировать LLM:
   * Действовать как бизнес-аналитик, готовящий краткую сводку для менеджера.
   * На основе предоставленных данных сгенерировать краткий (2-4 предложения) связный текст на русском языке.
   * В тексте нужно упомянуть ключевую метрику (например, GMV), ее динамику (рост/падение в %) и, возможно, кратко указать на динамику связанных метрик (например, заказы, сессии), если они сильно изменились.
   * Использовать только информацию из предоставленных данных.
4) Функция должна вызывать LLM с этим промптом и данными и возвращать сгенерированный текст сводки.
Предоставить краткое описание вашего выбора LLM, ее особенности и возможных улучшений для промпта или подхода.


### Подготовка к работе

In [1]:
import json
from huggingface_hub import login, logout, InferenceClient
from typing import Dict

In [2]:
summaries = {
    # 1. Сильный рост по всем фронтам (День)
    'daily_strong_growth' : {
      "period": "2023-11-15",
      "comparison_period": "2023-11-14",
      "metrics": {
        "GMV": {"value": 1625000, "change": 0.25}, # +25%
        "Заказы": {"value": 2400, "change": 0.20},  # +20%
        "Средний чек (AOV)": {"value": 677.08, "change": 0.042}, # +4.2%
        "Сессии": {"value": 60000, "change": 0.18},  # +18%
        "Конверсия (CR)": {"value": 0.04, "change": 0.017} # +1.7%
      }
    },
    
    # 2. Сильное падение по всем фронтам (День)
    'daily_strong_decline' : {
      "period": "2023-11-16",
      "comparison_period": "2023-11-15",
      "metrics": {
        "GMV": {"value": 980000, "change": -0.22},  # -22%
        "Заказы": {"value": 1800, "change": -0.18},  # -18%
        "Средний чек (AOV)": {"value": 544.44, "change": -0.049},# -4.9%
        "Сессии": {"value": 48000, "change": -0.15},  # -15%
        "Конверсия (CR)": {"value": 0.0375, "change": -0.035}# -3.5%
      }
    },
    
    # 3. Смешанная динамика: GMV растет, Заказы падают (День) -> Рост за счет AOV
    'daily_mixed_gmv_up_orders_down' : {
      "period": "2023-11-17",
      "comparison_period": "2023-11-16",
      "metrics": {
        "GMV": {"value": 1060000, "change": 0.082}, # +8.2%
        "Заказы": {"value": 1710, "change": -0.05},  # -5%
        "Средний чек (AOV)": {"value": 619.88, "change": 0.139}, # +13.9%
        "Сессии": {"value": 49000, "change": 0.021},  # +2.1%
        "Конверсия (CR)": {"value": 0.0349, "change": -0.069} # -6.9%
      }
    },
    
    # 4. Смешанная динамика: GMV падает, Заказы растут (День) -> Падение за счет AOV
    'daily_mixed_gmv_down_orders_up' : {
      "period": "2023-11-18",
      "comparison_period": "2023-11-17",
      "metrics": {
        "GMV": {"value": 985800, "change": -0.07},  # -7%
        "Заказы": {"value": 1778, "change": 0.04},  # +4%
        "Средний чек (AOV)": {"value": 554.44, "change": -0.106},# -10.6%
        "Сессии": {"value": 51000, "change": 0.041},  # +4.1%
        "Конверсия (CR)": {"value": 0.03486, "change": -0.001}# -0.1%
      }
    },
    
    # 5. Почти без изменений / Стабильность (День)
    'daily_flat' : {
      "period": "2023-11-19",
      "comparison_period": "2023-11-18",
      "metrics": {
        "GMV": {"value": 990000, "change": 0.004}, # +0.4%
        "Заказы": {"value": 1785, "change": 0.004}, # +0.4%
        "Средний чек (AOV)": {"value": 554.62, "change": 0.0},  # 0%
        "Сессии": {"value": 51100, "change": 0.002}, # +0.2%
        "Конверсия (CR)": {"value": 0.03493, "change": 0.002} # +0.2%
      }
    },
    
    # 6. Умеренный недельный рост
    'weekly_moderate_growth' : {
      "period": "Неделя 46, 2023",
      "comparison_period": "Неделя 45, 2023",
      "metrics": {
        "GMV": {"value": 7800000, "change": 0.09},  # +9%
        "Заказы": {"value": 13500, "change": 0.07},  # +7%
        "Средний чек (AOV)": {"value": 577.78, "change": 0.019}, # +1.9%
        "Сессии": {"value": 450000, "change": 0.05},  # +5%
        "Конверсия (CR)": {"value": 0.03, "change": 0.019} # +1.9%
      }
    },
    
    # 7. Недельное падение с аномалией в трафике (GMV/Заказы вниз, Сессии вверх)
    'weekly_decline_traffic_anomaly' : {
      "period": "Неделя 47, 2023",
      "comparison_period": "Неделя 46, 2023",
      "metrics": {
        "GMV": {"value": 7332000, "change": -0.06}, # -6%
        "Заказы": {"value": 12420, "change": -0.08}, # -8%
        "Средний чек (AOV)": {"value": 590.34, "change": 0.022}, # +2.2%
        "Сессии": {"value": 472500, "change": 0.05},  # +5% (аномалия!)
        "Конверсия (CR)": {"value": 0.0263, "change": -0.124} # -12.4%
      }
    },
    
    # 8. Фокус на конверсии (День) (GMV/Заказы стабильны, Сессии упали -> CR выросла)
    'daily_conversion_focus' : {
      "period": "2023-11-20",
      "comparison_period": "2023-11-19",
      "metrics": {
        "GMV": {"value": 995000, "change": 0.005}, # +0.5%
        "Заказы": {"value": 1790, "change": 0.003}, # +0.3%
        "Средний чек (AOV)": {"value": 555.87, "change": 0.002}, # +0.2%
        "Сессии": {"value": 48545, "change": -0.05}, # -5%
        "Конверсия (CR)": {"value": 0.03687, "change": 0.056} # +5.6%
      }
    },
    
    # 9. Рост за счет AOV (Неделя) (GMV умеренно вырос, Заказы слегка упали)
    'weekly_aov_driven_growth' : {
      "period": "Неделя 48, 2023",
      "comparison_period": "Неделя 47, 2023",
      "metrics": {
        "GMV": {"value": 7845240, "change": 0.07},  # +7%
        "Заказы": {"value": 12234, "change": -0.015}, # -1.5%
        "Средний чек (AOV)": {"value": 641.25, "change": 0.086}, # +8.6%
        "Сессии": {"value": 470000, "change": -0.005}, # -0.5%
        "Конверсия (CR)": {"value": 0.0260, "change": -0.01} # -1%
      }
    },
    
    # 10. Резкий всплеск (например, старт большой распродажи) (День)
    'daily_promo_spike' : {
      "period": "2023-11-24", # Черная пятница :)
      "comparison_period": "2023-11-23",
      "metrics": {
        "GMV": {"value": 2400000, "change": 0.6},  # +60%
        "Заказы": {"value": 3100, "change": 0.55},  # +55%
        "Средний чек (AOV)": {"value": 774.19, "change": 0.032}, # +3.2%
        "Сессии": {"value": 80000, "change": 0.4},  # +40%
        "Конверсия (CR)": {"value": 0.03875, "change": 0.107} # +10.7%
      }
    }
}

In [3]:
def dict_to_pretty_json(dictionary, indent=4):
    return json.dumps(dictionary, indent=indent, ensure_ascii=False)

In [4]:
summaries_list = [data for item, data in summaries.items()]
print(dict_to_pretty_json(summaries_list[0]))

{
    "period": "2023-11-15",
    "comparison_period": "2023-11-14",
    "metrics": {
        "GMV": {
            "value": 1625000,
            "change": 0.25
        },
        "Заказы": {
            "value": 2400,
            "change": 0.2
        },
        "Средний чек (AOV)": {
            "value": 677.08,
            "change": 0.042
        },
        "Сессии": {
            "value": 60000,
            "change": 0.18
        },
        "Конверсия (CR)": {
            "value": 0.04,
            "change": 0.017
        }
    }
}


### Функция, принимающая данные по KPI

In [5]:
def dict_to_pretty_str(data: dict) -> tuple[str, str, str]:
    period = data.get("period")
    comparison = data.get("comparison_period")
    metrics = data.get("metrics")
    
    metric_lines = []
    for metric_name, metric_data in metrics.items():
        value = metric_data["value"]
        change = metric_data["change"]
        sign = "+" if change > 0 else ""
        percent = round(change * 100, 1)
        metric_lines.append(f"{metric_name}: {value} ({sign}{percent}%)")
    
    metrics_text = "\n".join(metric_lines)
    
    return metrics_text, period, comparison

print(dict_to_pretty_str(summaries_list[0])[0])

GMV: 1625000 (+25.0%)
Заказы: 2400 (+20.0%)
Средний чек (AOV): 677.08 (+4.2%)
Сессии: 60000 (+18.0%)
Конверсия (CR): 0.04 (+1.7%)


In [6]:
# черновик промта
system_prompt = (
    "Ты — бизнес-аналитик. Проанализируй представленные KPI-метрики за период. "
    "Подготовь краткую текстовую сводку (1–3 предложения).\n\n"
    "Твоя задача — ясно и по делу сообщить, что изменилось:\n"
    "- Обязательно скажи как изменилась ключевая метрика - GMV (рост\падение) в процентах.\n"
    "- Отметь рост или падение других метрик, если оно превышает 5%. В противном случае вообще не упоминай их.\n"
    "- Делай выводы на основе представленных данных, не выдумывай ничего.\n\n"
    "Пиши на русском языке, деловым стилем, связно и кратко."
)

metrics_text, period, comparison = dict_to_pretty_str(summaries_list[0])
user_prompt = f"Сравни период {period} с {comparison}.\n\nПоказатели:\n{metrics_text}"

### Полный скрипт

In [7]:
token = input('Вставьте вашен токен для huggingface')

Вставьте вашен токен для huggingface 


In [8]:
if len(token) < 37:
    with open('token.txt', 'r') as f:
        token = f.readline()
login(token=token)

In [9]:

def summarize_kpi(data: dict) -> str:
    metrics_text, period, comparison = dict_to_pretty_str(data)
    
    system_prompt = (
        "Ты — бизнес-аналитик. Проанализируй представленные KPI-метрики за период. "
        "Подготовь краткую текстовую сводку (1–3 предложения).\n\n"
        "Твоя задача — ясно и по делу сообщить, что изменилось:\n"
        "- В первом предложении опиши общую тенденцию метрик, не вдаваясь в подробности, не упоминаю конкретные метрики (в среднем растут\практически не меняются\падают\смешанные показатели). НЕ БОЛЬШЕ ОДНОГО ПРЕДЛОЖЕНИЯ.\n"
        "- Во втором предложении обязательно скажи как изменилась ключевая метрика - GMV (рост\падение) в процентах. НЕ БОЛЬШЕ ОДНОГО ПРЕДЛОЖЕНИЯ.\n"
        "- В третьем предложении упомяни тендеции 1-2 других метрик с самыми значительными изменениями. Ни в коем случае не больше 1-2 метрик.  НЕ БОЛЬШЕ ОДНОГО ПРЕДЛОЖЕНИЯ..\n"
        "Не давай никаких лишних комментариев и интерпритаций данным.\n"
        "Не расшифровывай названия метрик и не пиши единицы измерения метрикам.\n"
        "Не делай сложных речевых оборотов.\n"
        "Делай выводы на основе представленных данных, не выдумывай ничего.\n\n"
        "Пиши на русском языке, деловым стилем, связно и кратко."
    )

    user_prompt = f"Сравни период {period} с {comparison}.\n\nПоказатели:\n{metrics_text}"


    messages = [{"role": "system", "content": system_prompt},
                 {"role": "user", "content": user_prompt}]
    
    client = InferenceClient("meta-llama/Meta-Llama-3-8B-Instruct")
    
    response = client.chat_completion(messages, max_tokens=100, temperature=0.2)
    
    return response.choices[0].message.content

### Результат работы скрипта

In [10]:
for summary in summaries_list:
    print(dict_to_pretty_str(summary)[0])
    print()
    print(summarize_kpi(summary))
    print()
    print('========================================')
    print()


GMV: 1625000 (+25.0%)
Заказы: 2400 (+20.0%)
Средний чек (AOV): 677.08 (+4.2%)
Сессии: 60000 (+18.0%)
Конверсия (CR): 0.04 (+1.7%)

Общая тенденция метрик характеризуется умеренным ростом. 
GMV увеличилась на 25,0% по сравнению с предыдущим периодом. 
Значительные изменения также наблюдаются в количестве заказов (+20,0%) и сессиях (+18,0%).


GMV: 980000 (-22.0%)
Заказы: 1800 (-18.0%)
Средний чек (AOV): 544.44 (-4.9%)
Сессии: 48000 (-15.0%)
Конверсия (CR): 0.0375 (-3.5%)

Общая тенденция метрик характеризуется снижением показателей.

GMV упала на 22,0% по сравнению с предыдущим периодом.

Значительные изменения наблюдаются в снижении среднего чека (AOV) на 4,9% и количества заказов на 18,0%.


GMV: 1060000 (+8.2%)
Заказы: 1710 (-5.0%)
Средний чек (AOV): 619.88 (+13.9%)
Сессии: 49000 (+2.1%)
Конверсия (CR): 0.0349 (-6.9%)

Общая тенденция метрик характеризуется смешанными показателями. 
GMV вырос на 8,2% по сравнению с предыдущим периодом. 
Значительные изменения наблюдаются в среднем че

### Финальные промты

In [11]:
system_prompt = (
    "Ты — бизнес-аналитик. Проанализируй представленные KPI-метрики за период. "
    "Подготовь краткую текстовую сводку (1–3 предложения).\n\n"
    "Твоя задача — ясно и по делу сообщить, что изменилось:\n"
    "- В первом предложении опиши общую тенденцию метрик, не вдаваясь в подробности, не упоминаю конкретные метрики (в среднем растут\практически не меняются\падают\смешанные показатели). НЕ БОЛЬШЕ ОДНОГО ПРЕДЛОЖЕНИЯ.\n"
    "- Во втором предложении обязательно скажи как изменилась ключевая метрика - GMV (рост\падение) в процентах. НЕ БОЛЬШЕ ОДНОГО ПРЕДЛОЖЕНИЯ.\n"
    "- В третьем предложении упомяни тендеции 1-2 других метрик с самыми значительными изменениями. Ни в коем случае не больше 1-2 метрик.  НЕ БОЛЬШЕ ОДНОГО ПРЕДЛОЖЕНИЯ..\n"
    "Не давай никаких лишних комментариев и интерпритаций данным.\n"
    "Не расшифровывай названия метрик и не пиши единицы измерения метрикам.\n"
    "Не делай сложных речевых оборотов.\n"
    "Делай выводы на основе представленных данных, не выдумывай ничего.\n\n"
    "Пиши на русском языке, деловым стилем, связно и кратко."
)
print('Системный промт')
print()
print(system_prompt)

Системный промт

Ты — бизнес-аналитик. Проанализируй представленные KPI-метрики за период. Подготовь краткую текстовую сводку (1–3 предложения).

Твоя задача — ясно и по делу сообщить, что изменилось:
- В первом предложении опиши общую тенденцию метрик, не вдаваясь в подробности, не упоминаю конкретные метрики (в среднем растут\практически не меняются\падают\смешанные показатели). НЕ БОЛЬШЕ ОДНОГО ПРЕДЛОЖЕНИЯ.
- Во втором предложении обязательно скажи как изменилась ключевая метрика - GMV (рост\падение) в процентах. НЕ БОЛЬШЕ ОДНОГО ПРЕДЛОЖЕНИЯ.
- В третьем предложении упомяни тендеции 1-2 других метрик с самыми значительными изменениями. Ни в коем случае не больше 1-2 метрик.  НЕ БОЛЬШЕ ОДНОГО ПРЕДЛОЖЕНИЯ..
Не давай никаких лишних комментариев и интерпритаций данным.
Не расшифровывай названия метрик и не пиши единицы измерения метрикам.
Не делай сложных речевых оборотов.
Делай выводы на основе представленных данных, не выдумывай ничего.

Пиши на русском языке, деловым стилем, связно и

In [12]:

metrics_text, period, comparison = dict_to_pretty_str(summaries_list[0])
user_prompt = f"Сравни период {period} с {comparison}.\n\nПоказатели:\n{metrics_text}"
print('Промт \'юзера\'')
print()
print(user_prompt)

Промт 'юзера'

Сравни период 2023-11-15 с 2023-11-14.

Показатели:
GMV: 1625000 (+25.0%)
Заказы: 2400 (+20.0%)
Средний чек (AOV): 677.08 (+4.2%)
Сессии: 60000 (+18.0%)
Конверсия (CR): 0.04 (+1.7%)


In [13]:
logout()

### Объснение

**Выбор модели**:
- Выбрана модель meta-llama/Meta-Llama-3-8B-Instruct, так как является базовой и самой доступной. При этом тестирование модели показало, что она вполне подходит для этой задачи.
- Используется библиотека huggingface_hub, так как позволяет очень легко использовать hf inference API.

**Составление промта**:
- Было решено четко структурировать ответ с помощью промта, так ответы стали стабильнее и гораздо лучше. При этом количество предложений жестко задается как 3, что не совсем противоречит условию при этом улучшает результат. 
- Промт содержит много негативных инструкций (как не делать), все они были добавлены после тестирования ответов
    
      Модель иногда писала большие сложные предложения и не укладывалась в свой максимум токенов.
      Модель могла придумывать расшифровку аббревиатурам или писать, что средний чек в долларах, хотя это не указано.
      Модель могла начать писать, что бизнес хорошо растет и какую-то такую интерпретацию данным.
      Модель могла писать в несколько предложений каждый пункт, поэтому решено дополнительно укзаать в конце ограничение.
      Уточнены рамки пунктов сводки.
  
- Основной промт подан как системный, а данные в пользовательский. Судя по документации общие рекомендации так лучше соблюдаются.

**Параметры модели**:
- Выбрана низкая температура в 0.2, так как условие не требует какого-то глубоко анализа и креативности от модели, скорее наоборот.
- Выбран максимум токенов ответа в 100, чтобы экономить бесплатные токены

**Идеи для улучшения**:
- Добавить автоматическое определение значимости изменений, например, от 5-ти процентов
- Использовать шаблоны вывода моделей, чтобы четче структурировать ответ